In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import pickle as pkl
import json
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_predict

In [2]:
DATASET_TAGS = ['PBMC1']
TOOLS = ['seurat', 'scanpy'] #, 'monocle', 'scvi-tools', 'COTAN']
TUNING = 'celltypist' # or 'antibody'
N_MARKERS = 500
RANDOM_MARKERS_TRIALS = 10
STEP = 100
START = 100
cluster_names = ['Classical monocytes', 'Naive B cells'] # should be in json files "mapping.json"
clf = RandomForestClassifier()

In [3]:
for dataset in DATASET_TAGS:
	print(f"# Processing dataset {dataset}")
	data_path = f"./data/{dataset}/filtered/10X/"
	if TUNING == "celltypist":
		labels_path = f"./data/{dataset}/{TUNING}/{TUNING}_labels.csv"
	else:
		labels_path = f"./data/{dataset}/{TUNING}_annotation/{TUNING}_labels.csv"
	# load gene expression matrix
	adata = sc.read_10x_mtx(
		data_path,
		var_names='gene_symbols',
		cache=False
	)
	# load true labels
	y_df = pd.read_csv(labels_path, index_col=0)
	y_df.index = y_df.index.str.replace("-1", "")
	y_df = pd.DataFrame(adata.obs_names, columns=["cell"]).join(y_df, on="cell")
	y = np.array(y_df['cluster.ids']).astype(int)
	# load ground truth mapping
	gt_mapping_path = f"./data/{dataset}/{TUNING}/{TUNING}_mapping.csv"
	gt_mapping_df = pd.read_csv(gt_mapping_path, index_col=1)
	gt_mapping_dict = {}
	for index, row in gt_mapping_df.iterrows():
		gt_mapping_dict[index] = row['id']
	# binarize labels
	y_ovr = {}
	for cluster_name, cluster_id in gt_mapping_dict.items():
		if cluster_name in cluster_names:
			y_ovr[cluster_name] = np.array(y==cluster_id, dtype=int)
	# compute cluster sizes to use as weights for averaging f1
	weights = np.zeros(len(y_ovr))
	for i, gt_cluster_name in enumerate(cluster_names):
		weights[i] = y_ovr[gt_cluster_name].sum()
	
	# evaluate each tool
	print("## Training 1 vs Rest RF on markers")
	f1 = {}
	f1_subsets = {}
	for tool in TOOLS:
		print(f"Tool {tool}")
		f1[tool] = {}
		f1_subsets[tool] = {}
		f1_tool = []
		# load tool markers
		markers_path = f"./data/{dataset}/{tool}/{TUNING}/markers.csv"
		markers_df = pd.read_csv(markers_path)
		# load tool mapping
		with open(f'./data/{dataset}/{tool}/{TUNING}/mapping.json', 'r') as file:
			tool_mapping = json.load(file)
		
		# train binary classifiers for each cluster using markers
		for gt_cluster_name, tool_cluster_id in tool_mapping.items():
			# train using all markers
			print("Training with all markers")
			markers = markers_df[(markers_df['cluster']==tool_cluster_id)]['gene'].values
			X_markers = adata[:, markers].X.toarray()
			y_pred = cross_val_predict(clf, X_markers, y_ovr[gt_cluster_name], cv=5)
			f1_tool_cluster = f1_score(y_ovr[gt_cluster_name], y_pred)
			f1[tool][gt_cluster_name] = f1_tool_cluster
			f1_tool.append(f1_tool_cluster)
			print(f"Cluster '{gt_cluster_name}' done")
			
			# train using subsets of markers
			f1_subsets[tool][gt_cluster_name] = []
			for i in range(START, N_MARKERS, STEP):
				print(f"Training with {i} markers")
				markers = markers_df[
					(markers_df['cluster']==tool_cluster_id) &
					(markers_df['rank']<=i)
				]['gene'].values
				X_markers = adata[:, markers].X.toarray()
				y_pred = cross_val_predict(clf, X_markers, y_ovr[gt_cluster_name], cv=5)
				f1_subset_tool_cluster = f1_score(y_ovr[gt_cluster_name], y_pred)
				f1_subsets[tool][gt_cluster_name].append(f1_subset_tool_cluster)
			
			# append score using all markers
			f1_subsets[tool][gt_cluster_name].append(f1_tool_cluster)
		
		f1_weighted_avg = np.average(f1_tool, weights=weights)
		f1[tool]['avg'] = f1_weighted_avg 
		var = np.average((f1_tool-f1_weighted_avg)**2, weights=weights)
		std = np.sqrt(var)
		f1[tool]['std'] = std
	
	# evaluate random markers
	print(f"## Training 1 vs Rest RF on a random subset of {N_MARKERS} features")
	f1['random_markers'] = {}
	f1_random = []
	for gt_cluster_name in tool_mapping.keys():
		f1_trials = []
		for i in range(RANDOM_MARKERS_TRIALS):
			print(f"Trial {i+1}/{RANDOM_MARKERS_TRIALS}")
			X_sample = adata[:, np.random.choice(adata.shape[1], N_MARKERS, replace=False)].X.toarray()
			y_pred = cross_val_predict(clf, X_sample, y_ovr[gt_cluster_name], cv=5)
			f1_random_cluster = f1_score(y_ovr[gt_cluster_name], y_pred)
			f1_trials.append(f1_random_cluster)
		f1_avg = np.average(f1_trials)
		f1_std = np.std(f1_trials)
		f1['random_markers'][gt_cluster_name] = str("{:.6f}".format(f1_avg))+' +/- '+str("{:.6f}".format(f1_std))
		f1_random.append(f1_avg)
	
	f1_weighted_avg = np.average(f1_random, weights=weights)
	f1['random_markers']['avg'] = f1_weighted_avg 
	var = np.average((f1_random-f1_weighted_avg)**2, weights=weights)
	std = np.sqrt(var)
	f1['random_markers']['std'] = std

	# save classification results
	f1_df = pd.DataFrame.from_dict(f1)
	display(f1_df)
	f1_df.to_csv(f"./data/{dataset}/rf_f1_{TUNING}.csv")

	# save f1 computed on subsets of markers
	with open(f"./data/{dataset}/rf_f1_{TUNING}_increasing.pkl", 'wb') as fp:
		pkl.dump(f1_subsets, fp)

# Processing dataset PBMC1
## Training 1 vs Rest RF on markers
Tool seurat
Training with all markers


/Users/irenetesta/anaconda3/envs/chl/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


Cluster 'Classical monocytes' done
Training with 100 markers


/Users/irenetesta/anaconda3/envs/chl/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


Training with 200 markers


...

Trial 9/10


/Users/irenetesta/anaconda3/envs/chl/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


Trial 10/10


/Users/irenetesta/anaconda3/envs/chl/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


,seurat,scanpy,random_markers
Classical monocytes,0.978116,0.976589,0.947857 +/- 0.008753
Naive B cells,0.845878,0.873646,0.358352 +/- 0.191403
avg,0.961894,0.963960,0.875541
std,0.043382,0.033771,0.193394
